# Fine-tune PhoBERT cho Claim Detection
Binary classification: claim vs non-claim

## 1. Cài đặt thư viện

In [ ]:
!pip install transformers datasets accelerate -q
!pip install scikit-learn -q

## 2. Import và kiểm tra GPU

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import warnings
warnings.filterwarnings('ignore')

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 3. Cấu hình

In [ ]:
# Cấu hình
MODEL_NAME = "vinai/phobert-base"
MAX_LENGTH = 128
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_EPOCHS = 5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01

# Đường dẫn dataset - CẬP NHẬT THEO KAGGLE
TRAIN_PATH = "/kaggle/input/claim-detection-dataset/claim_detection_train.jsonl"
VAL_PATH = "/kaggle/input/claim-detection-dataset/claim_detection_val.jsonl"
TEST_PATH = "/kaggle/input/claim-detection-dataset/claim_detection_test.jsonl"

# Labels
LABEL2ID = {"non-claim": 0, "claim": 1}
ID2LABEL = {0: "non-claim", 1: "claim"}

## 4. Load dữ liệu

In [ ]:
def load_jsonl(file_path):
    """Load file JSONL"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Load data
train_df = load_jsonl(TRAIN_PATH)
val_df = load_jsonl(VAL_PATH)
test_df = load_jsonl(TEST_PATH)

print(f"Train: {len(train_df)} samples")
print(f"Val: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

print("\nTrain label distribution:")
print(train_df['label'].value_counts())

print("\nSample data:")
train_df.head()

## 5. Tạo Dataset class

In [ ]:
class ClaimDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['text']
        label = LABEL2ID[row['label']]
        
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

## 6. Load PhoBERT

In [ ]:
print(f"Loading {MODEL_NAME}...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
    id2label=ID2LABEL,
    label2id=LABEL2ID
)

print(f"✓ Model loaded")
print(f"Parameters: {model.num_parameters():,}")

## 7. Tạo datasets

In [ ]:
train_dataset = ClaimDataset(train_df, tokenizer, MAX_LENGTH)
val_dataset = ClaimDataset(val_df, tokenizer, MAX_LENGTH)
test_dataset = ClaimDataset(test_df, tokenizer, MAX_LENGTH)

print(f"Train dataset: {len(train_dataset)}")
print(f"Val dataset: {len(val_dataset)}")
print(f"Test dataset: {len(test_dataset)}")

# Test
sample = train_dataset[0]
print(f"\nSample input_ids shape: {sample['input_ids'].shape}")
print(f"Sample label: {sample['labels']}")

## 8. Định nghĩa metrics

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='binary')
    recall = recall_score(labels, preds, average='binary')
    f1 = f1_score(labels, preds, average='binary')
    
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

## 9. Training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

print("Training arguments configured")

## 10. Khởi tạo Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

print("✓ Trainer initialized")

## 11. Training

In [ ]:
print("="*60)
print("BẮT ĐẦU TRAINING")
print("="*60)

trainer.train()

## 12. Đánh giá trên validation set

In [ ]:
print("\n" + "="*60)
print("ĐÁNH GIÁ TRÊN VALIDATION SET")
print("="*60)

eval_results = trainer.evaluate()
print("\nKết quả:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

## 13. Đánh giá trên test set

In [ ]:
print("\n" + "="*60)
print("ĐÁNH GIÁ TRÊN TEST SET")
print("="*60)

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids

print("\nClassification Report:")
print(classification_report(labels, preds, target_names=['non-claim', 'claim']))

print("\nMetrics:")
print(f"  Accuracy: {accuracy_score(labels, preds):.4f}")
print(f"  Precision: {precision_score(labels, preds):.4f}")
print(f"  Recall: {recall_score(labels, preds):.4f}")
print(f"  F1-Score: {f1_score(labels, preds):.4f}")

## 14. Lưu model

In [ ]:
# Lưu model
model.save_pretrained('./phobert_claim_detection')
tokenizer.save_pretrained('./phobert_claim_detection')

print("✓ Model saved to ./phobert_claim_detection")

## 15. Test inference

In [ ]:
def predict(text):
    """Dự đoán một câu"""
    model.eval()
    
    encoding = tokenizer(
        text,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    device = next(model.parameters()).device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.softmax(outputs.logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
    
    label = ID2LABEL[pred]
    confidence = probs[0][pred].item()
    
    return label, confidence

# Test
test_sentences = [
    "Việt Nam có 54 dân tộc anh em",
    "Dân số Hà Nội năm 2023 đạt 8 triệu người",
    "Bạn có biết điều này không?",
    "Tôi nghĩ rằng đây là quyết định đúng đắn",
    "Hãy xem xét kỹ vấn đề này",
    "TP HCM là thành phố lớn nhất Việt Nam",
]

print("\nTest predictions:")
print("="*60)
for sent in test_sentences:
    label, conf = predict(sent)
    print(f"\n{sent}")
    print(f"  → {label} (confidence: {conf:.3f})")

## 16. Nén model để download

In [ ]:
!zip -r phobert_claim_detection.zip phobert_claim_detection/
print("\n✓ Model đã được nén thành phobert_claim_detection.zip")
print("Download từ Output tab của Kaggle notebook")

## Hoàn thành! 🎉